<a href="https://colab.research.google.com/github/Edwin372/Thesis/blob/main/Devign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers 
!pip install datasets 
!pip install tokenizers
!pip install accelerate
!pip install wandb
!pip install sentence_transformers
!pip install torchmetrics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import huggingface_hub

# huggingface_hub.login()

In [3]:
from sklearn.utils import compute_class_weight
import torch.nn as nn
import torch
from typing import Optional, Tuple, Union, List
from accelerate import Accelerator
import wandb
from transformers.modeling_outputs import TokenClassifierOutput
from torchmetrics.classification.f_beta import F1Score, BinaryF1Score
from torchmetrics.classification.precision_recall import Precision, Recall, BinaryPrecision, BinaryRecall 
from torchmetrics.classification.accuracy import BinaryAccuracy, Accuracy
from tqdm.auto import tqdm
import numpy as np
import json
import pandas as pd
# from torch_geometric.data import Data
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer, models
from datasets import Dataset, ClassLabel


def add_token(tokenizer, model=None):
    tokenizer.add_tokens([f'VAR_{i}' for i in range(300)])
    tokenizer.add_tokens([f'FUNC_{i}' for i in range(100)])
    tokenizer.add_tokens([f'CLASS_{i}' for i in range(100)])

    if model is not None:
        model.resize_token_embeddings(len(tokenizer))
        
    return model, tokenizer 

def add_special_token(tokenizer, model=None):
    tokenizer.add_special_tokens([f'VAR_{i}' for i in range(300)])
    tokenizer.add_special_tokens([f'FUNC_{i}' for i in range(100)])
    tokenizer.add_special_tokens([f'CLASS_{i}' for i in range(100)])

    if model is not None:
        model.resize_token_embeddings(len(tokenizer))
        
    return model, tokenizer 



def get_model_and_tokenizer(ckt:str, use_special_token=False):
  from transformers import AutoTokenizer, AutoModel
  model = AutoModel.from_pretrained(ckt, output_attentions=True)
  tokenizer = AutoTokenizer.from_pretrained(ckt)
  if not use_special_token:
    model, tokenizer = add_token(tokenizer, model)
  else:
    model, tokenizer = add_special_token(tokenizer, model)
  return model, tokenizer 

In [4]:
from datasets import load_dataset
import huggingface_hub

huggingface_hub.login()


In [5]:
ds = load_dataset('EddieChen372/devign_with_vul_lines')
ds

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'func', 'target', 'project', 'commit_id', 'func_clean', 'vul_lines'],
        num_rows: 2732
    })
    train: Dataset({
        features: ['id', 'func', 'target', 'project', 'commit_id', 'func_clean', 'vul_lines'],
        num_rows: 21854
    })
    test: Dataset({
        features: ['id', 'func', 'target', 'project', 'commit_id', 'func_clean', 'vul_lines'],
        num_rows: 2732
    })
})

In [6]:
def is_statement_vulnerable(statement, lines, vul_lines):
  
  if len(vul_lines) == 0: return 0 

  keywords = {' ', '', '\n','{', '}', 'if', 'else', 'for', 'while', 'do'}
  tokens = set(statement.strip().split(' '))

  if tokens.issubset(keywords):
    return 0
  
  for line in lines:
    if line in vul_lines: 
      return 1 
  return 0 
  
  

def get_statement(sample):
  func = sample['func_clean']
  vul_lines = sample['vul_lines']['line_no']
  lines = enumerate(func.split('\n'), 1)
  
  bracket_queue = [] 
  statements = {
      'lines': [],
      'line_no': [],
      'label': []
  }
  cur_statement = ''
  cur_lines = [] 
  is_keyword_statement = False 

  for line_no, line in lines:
    if line.strip() != '':
      cur_statement += ('\n'+line.strip()) if cur_statement != '' else line.strip() 
      cur_lines.append(line_no)
      if ';' in line or '{' in line or '}' in line:
        statements['lines'].append(cur_statement) 
        statements['line_no'].append(cur_lines)
        statements['label'].append(is_statement_vulnerable(cur_statement, cur_lines, vul_lines))
        cur_statement = ''
        cur_lines = []
      

  sample['lines'] = statements['lines']
  sample['label'] = statements['label']
  sample['line_no'] = statements['line_no']

  return sample
        

In [7]:
sample = ds['train'][5]

In [8]:
new_sample = get_statement(sample)

In [9]:
new_sample['vul_lines']

{'code': ['                        (frame->data[1][3+i*4]<<24)'],
 'line_no': [227]}

In [11]:
new_sample['lines']

['static int dds_decode(AVCodecContext *avctx, void *data,\nint *got_frame, AVPacket *avpkt)\n{',
 'DDSContext *ctx = avctx->priv_data;',
 'GetByteContext *gbc = &ctx->gbc;',
 'AVFrame *frame = data;',
 'int mipmap;',
 'int ret;',
 'ff_texturedsp_init(&ctx->texdsp);',
 'bytestream2_init(gbc, avpkt->data, avpkt->size);',
 'if (bytestream2_get_bytes_left(gbc) < 128) {',
 'av_log(avctx, AV_LOG_ERROR, "Frame is too small (%d).\\n",\nbytestream2_get_bytes_left(gbc));',
 'return AVERROR_INVALIDDATA;',
 '}',
 "if (bytestream2_get_le32(gbc) != MKTAG('D', 'D', 'S', ' ') ||\nbytestream2_get_le32(gbc) != 124) {",
 'av_log(avctx, AV_LOG_ERROR, "Invalid DDS header.\\n");',
 'return AVERROR_INVALIDDATA;',
 '}',
 'bytestream2_skip(gbc, 4);',
 'avctx->height = bytestream2_get_le32(gbc);',
 'avctx->width  = bytestream2_get_le32(gbc);',
 'ret = av_image_check_size(avctx->width, avctx->height, 0, avctx);',
 'if (ret < 0) {',
 'av_log(avctx, AV_LOG_ERROR, "Invalid image size %dx%d.\\n",\navctx->width, avc

In [12]:
ds = ds.map(get_statement)

  0%|          | 0/2732 [00:00<?, ?ex/s]

  0%|          | 0/21854 [00:00<?, ?ex/s]

  0%|          | 0/2732 [00:00<?, ?ex/s]

In [44]:

from transformers import AutoTokenizer, RobertaModel, BertPreTrainedModel,BertModel
from datasets import Dataset, ClassLabel
from transformers import BertConfig
from transformers.modeling_outputs import TokenClassifierOutput, SequenceClassifierOutput

accelerator = Accelerator(gradient_accumulation_steps=4)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
accelerator.state.device = device

class BertForLineClassification(BertPreTrainedModel):
    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        inputs_embeds:torch.Tensor,
        labels:torch.Tensor=None,
        class_weight=None,
        output_attentions: Optional[bool] = True,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ): 

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None and class_weight is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weight, reduce='mean')
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

class BertForFuncClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        inputs_embeds:torch.Tensor,
        labels:torch.Tensor=None,
        class_weight=None,
        output_attentions: Optional[bool] = True,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ): 

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        print(pooled_output.shape)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None and class_weight is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weight, reduce='mean')
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


class Conan(nn.Module):

    def __init__(self, embs_model_ckt:str, num_labels:int, num_hidden_layers=6, max_lines=1024, func_cls=False):
        super(Conan, self).__init__()
        self.embs_model, self.tokenizer = get_model_and_tokenizer(embs_model_ckt)
        self.embs_model = self.embs_model.to(device)
        config = BertConfig(
            vocab_size=1,
            hidden_size=self.embs_model.config.hidden_size,
            num_hidden_layers=num_hidden_layers,
            max_position_embeddings=max_lines,
            num_labels=num_labels
        )
        self.func_cls = func_cls
        if self.func_cls:
            self.model = BertForFuncClassification(config=config).to(device)
        else:
            self.model = BertForLineClassification(config=config).to(device)


    def cls_pooling(self, model_output):
      return model_output.last_hidden_state[:, 0]

    def forward(self, lines, labels:torch.LongTensor=None, class_weight:torch.FloatTensor=None, masked=True):
        inputs = self.tokenizer(
            lines if not self.func_cls else [self.tokenizer.cls_token] + lines, 
            padding=True, 
            return_tensors='pt',
            truncation=True,
            max_length=128
        )
        if class_weight is not None and labels is not None:
            class_weight = class_weight.to(device)
            labels = labels.to(device)

        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)

        if masked:
          attention_mask = torch.where(input_ids < self.tokenizer.vocab_size, attention_mask, 0).to(device)
        

        embs_out = self.embs_model(input_ids=input_ids, attention_mask=attention_mask) 
        line_embeddings = self.cls_pooling(embs_out).unsqueeze(0)

        return self.model( 
            inputs_embeds=line_embeddings.to(device),
            labels=labels,
            class_weight=class_weight,
        )

    def save_pretrained(self, name):
        self.tokenzier.save_pretrained(f'{name}/')
        self.embs_model.save_pretrained(f'{name}/embs')
        self.model.save_pretrained(f'{name}/classifier')
        
    def from_pretrained(self, name):
        from transformers import AutoModel, AutoTokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(f'{name}/embs')
        self.embs_model = AutoModel.from_pretrained(f'{name}/embs')
        self.model = BertForLineClassification.from_pretrained(f'{name}/classifier')
        
class ConanTrainerConfig():
    def __init__(self, ds, epoch_num, model_ckt, learning_rate=1e-5, masked=True ,device='cpu', func_cls=False, save_each=10) -> None:
        self.learning_rate = learning_rate
        self.device = device
        self.ds = ds
        self.epoch_num = epoch_num
        y = np.concatenate(self.ds['train'][:]['label'])
        self.class_weight = torch.FloatTensor(compute_class_weight('balanced', y=y, classes=np.unique(y)))
        self.model = Conan(model_ckt, num_labels=len(self.class_weight), func_cls=func_cls)
        self.func_cls = func_cls
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)
        self.scheduler = torch.optim.lr_scheduler.LinearLR(self.optimizer)
        self.masked = masked
        self.save_each = save_each 
        self.num_classes = len(self.class_weight)


class ConanTrainer():
    def __init__(self, 
                config, 
                log=True
        ):
        self.config = config  
        self.train_ds = self.config.ds['train']
        self.test_ds = self.config.ds['test']
        self.valid_ds = self.config.ds['validation']
        self.device = self.config.device
        self.model = self.config.model
        self.optimizer = self.config.optimizer 
        self.scheduler = self.config.scheduler 
        self.class_weight = self.config.class_weight
        self.num_classes = self.config.num_classes 
        self.masked = self.config.masked
        self.log = log

    def run_train(self):
        self.model.train()
        total_step = len(self.train_ds)
        total_loss = 0
        for i in tqdm(range(total_step)):  
            try:
                self.optimizer.zero_grad()
                sample = self.train_ds[i]
                lines = sample['lines'] 
                labels = torch.LongTensor([sample['target']]) if self.config.func_cls else torch.LongTensor(sample['label']) 
                with accelerator.accumulate(self.model):
                    out = self.model(sample['lines'], labels=labels, class_weight=self.class_weight, masked=self.masked)
                    total_loss += out.loss.item() 
                    accelerator.backward(out.loss)
                    self.optimizer.step()
                    self.scheduler.step()
            except:
                print(len(sample['lines']))
                continue
        return total_loss/total_step
 
    def run_test(self, threshold=0.5, average='macro', eval_valid=False):
        self.model.eval()
        if self.num_classes > 2:
            f1 = F1Score(task='multiclass', num_classes=self.num_classes, average=average, threshold=threshold).to(self.device)
            precision = Precision('multiclass', num_classes=self.num_classes, average=average, threshold=threshold).to(self.device)
            recall = Recall('multiclass' ,num_classes=self.num_classes, average=average, threshold=threshold).to(self.device)
            acc = Accuracy('multiclass' ,num_classes=self.num_classes, average=average, threshold=threshold).to(self.device)
        else:
            f1 = BinaryF1Score(threshold=threshold).to(self.device)
            precision = BinaryPrecision(threshold=threshold).to(self.device)
            recall = BinaryRecall(threshold=threshold).to(self.device)
            acc = BinaryAccuracy(threshold=threshold).to(self.device)

        preds = []
        targets = []
        total_loss = 0
        ds = self.test_ds if eval_valid else self.valid_ds

        with torch.no_grad():
            for i in tqdm(range(len(ds))):  
                try:
                    sample = ds[i]
                    labels = torch.LongTensor([sample['target']]) if self.config.func_cls else torch.LongTensor(sample['label']) 
                    # print(labels)
                    out = self.model(sample['lines'], labels ,class_weight=self.class_weight)
                    total_loss += out.loss.item()
                    # y = torch.LongTensor([sample['target']])
                    # print(out.logits.shapejj)
                    preds.append(out.logits[0].argmax(dim=1))
                    targets.append(labels)
                except:
                    print(len(sample['lines']))
    
        preds = torch.concatenate(preds).to(self.device)
        targets = torch.concatenate(targets).to(self.device)

        if average == 'none':
            print(f1(preds, targets))
            print(precision(preds, targets)) 
            print(recall(preds, targets))
            print(acc(preds, targets))
            return 

        return  (
            total_loss/len(ds),
            f1(preds, targets).item(), 
            precision(preds, targets).item(), 
            recall(preds, targets).item(),
            acc(preds, targets).item()
        )
    
    def fit(self, name ):
        self.model, self.scheduler, self.optimizer = accelerator.prepare(
            self.model, self.scheduler, self.optimizer 
        )

        if self.log:
            wandb.init(
                name=name,
                project=f'Thesis graph classifiers',
                entity='eddiechen372',
                reinit=True,
                config={
                    'architecture': 'codebert'
                }
            )
        model_num_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        if self.log: 
            wandb.log({'embs num params': model_num_params}) 
        for epoch in range(1, self.config.epoch_num + 1):
            accelerator.free_memory()
            train_loss = self.run_train()
            eval_loss, test_f1, test_p, test_r, test_acc = self.run_test()
            print(f'train loss: {train_loss}', f'eval loss: {eval_loss}')
            print(f'Epoch: {epoch:03d}, test precision: {test_p:.4f}, test recall: {test_r:.4f}, Test f1: {test_f1:.4f} , Test acc: {test_acc:.4f}')

            if self.log: 
                wandb.log({
                    'epoch': epoch, 
                    'test f1': test_f1, 
                    'test precision': test_p, 
                    'test recall': test_r, 
                    'test acc': test_acc, 
                    'train loss': train_loss,
                    'eval loss': eval_loss
                })
        
    def evaluate(self):
        self.model, self.scheduler, self.optimizer = accelerator.prepare(
            self.model, self.scheduler, self.optimizer
        )
        num_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print('model num params:', num_params) 
        self.run_test(average='none')

         
    def save(self, name):
        self.model.embs_model.save_pretrained(f'{name}/embs')
        self.model.model.save_pretrained(f'{name}/classifier')
        self.model.tokenizer.save_pretrained(f'{name}/classifier')

    def load(self, name):
        from transformers import AutoModel, AutoTokenizer
        self.model.embs_model = AutoModel.from_pretrained(f'{name}/embs')
        self.model.model = BertForLineClassification.from_pretrained(f'{name}/classifier')
        self.model.tokenizer = AutoTokenizer.from_pretrained(f'{name}/classifier')
        

In [45]:
new_ds = ds.filter(lambda x: 
          (x['target']==False and len(x['vul_lines']['code']) ==0 ) or
          (x['target']==True and len(x['vul_lines']['code'])> 0 )
          )
new_ds

DatasetDict({
    validation: Dataset({
        features: ['id', 'func', 'target', 'project', 'commit_id', 'func_clean', 'vul_lines', 'lines', 'label', 'line_no'],
        num_rows: 2732
    })
    train: Dataset({
        features: ['id', 'func', 'target', 'project', 'commit_id', 'func_clean', 'vul_lines', 'lines', 'label', 'line_no'],
        num_rows: 21854
    })
    test: Dataset({
        features: ['id', 'func', 'target', 'project', 'commit_id', 'func_clean', 'vul_lines', 'lines', 'label', 'line_no'],
        num_rows: 2732
    })
})

In [46]:
model_ckt = 'sentence-transformers/all-miniLM-l6-v2' 
# model_ckt = 'neulab/codebert-python' 
# model_ckt = 'neulab/codebert-python'
model = Conan(model_ckt, num_labels=2)
config = ConanTrainerConfig(
    ds=ds,
    model_ckt=model_ckt,
    epoch_num=10,
    learning_rate=1e-5,
    device=device,
    masked=False,
    func_cls=False
    save_each=10 
)

In [47]:
config.class_weight
trainer = ConanTrainer(config=config, log=False)

In [48]:
trainer.evaluate()

model num params: 41035010


  0%|          | 0/2732 [00:00<?, ?it/s]

2455
1034
tensor(0.0625, device='cuda:0')
tensor(0.0489, device='cuda:0')
tensor(0.0865, device='cuda:0')
tensor(0.9259, device='cuda:0')


In [ ]:
trainer

In [ ]:
trainer.evaluate()
trainer.save('devign')
trainer.load('devign')
trainer.evaluate()